In [9]:
import openai
import os
import json
import subprocess
pat = os.getenv('OPENAI_API_KEY')

In [10]:
# The client uses OpenAI Library version 0.27.0. To use 1.0.0, uncomment the line below
client = openai.OpenAI(api_key=pat)

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.1, # this is the degree of randomness of the model's output
    )

    return response.choices[0].message.content

In [11]:
# load unit tests
unittest_name = "addition_test"

with open(f"./{unittest_name}.py", "r") as file:
    unittest_code = file.read()

In [12]:
# initial prompt

lang = "Python"


initial_prompt = prompt = f"""

You're a {lang} software developer. You will write code to make the unit tests given in the triple backticks passed. You will use the strategies of test driven development to do this. Provide the code to make the first test pass.

Perform the following actions:

1. Summarize what the code is intended to do given the first test.
2. List the names of the functions that this test is covering.
3. Provide the code to make this test pass.
4. Output the above steps in a JSON object in the following format: 
{{
    "test_summary": "1", 
    "function_names": "2",
    "code": "3"}}

tests:
```{unittest_code}```

"""

In [13]:
response = get_completion(prompt)
json_response = json.loads(response)
print(response)

{
    "test_summary": "The code is intended to test the addition function with positive integers.",
    "function_names": "add",
    "code": "def add(a, b):\n    return a + b"
}


In [14]:
# output response string to a python file. Need to keep track of how many files we've created

filename = unittest_name.strip("_test")
print(filename)
with open(f"{filename}.py", "w") as file:
    file.write(json_response["code"])

addition


In [15]:
# run tests against the response
command = ["python3", "-m", "unittest", unittest_name]
output = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
print(output.stderr)

..F...
FAIL: test_add_non_numbers (addition_test.TestAddFunction.test_add_non_numbers)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/Users/jtmcgoffin/Documents/hackathon-2-22-24/Hackathon-2-22-24/projects/swarm-ai/addition_test.py", line 23, in test_add_non_numbers
    with self.assertRaises(TypeError):
AssertionError: TypeError not raised

----------------------------------------------------------------------
Ran 6 tests in 0.001s

FAILED (failures=1)



In [16]:
# iterate until all tests pass
def get_success(test_output):
    return "OK" in test_output

def get_iteration_prompt(test_feedback):
    return f"""

    Some or all of the tests did not pass. The feedback from the test runner is in the triple backticks. Using that feedback, iterate on the code you previously provided to make the failing tests pass while maintaining that the passing tests continue to pass. Make sure that any non-test errors are addressed first and remove any unused imports. Perform the following actions:

    1. Summarize what the code is intended to do given the tests.
    2. List the names of the functions that these tests are covering.
    3. Provide the code.
    4. 4. Output the above steps in a JSON object in the following format: 
    {{
    "test_summary": "1", 
    "function_names": "2",
    "code": "3"}}

    test feedback:
    ```{test_feedback}```

    """

num_iterations = 0

while(not get_success(output.stderr)):
    num_iterations += 1
    print(f"iteration {num_iterations}")
    response = get_completion(get_iteration_prompt(output.stderr))
    json_response = json.loads(response)

    # output response string to a python file. Need to keep track of how many files we've created
    with open(f"{filename}.py", "w") as file:
        file.write(json_response["code"])

    # run tests against the response
    command = ["python3", "-m", "unittest", unittest_name]
    output = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    print(output)

iteration 1
CompletedProcess(args=['python3', '-m', 'unittest', 'addition_test'], returncode=0, stdout='', stderr='......\n----------------------------------------------------------------------\nRan 6 tests in 0.000s\n\nOK\n')
